<a href="https://colab.research.google.com/github/RTae/Shopee-Code-League/blob/master/week1/Order_Brushing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime

In [0]:
df_order = pd.DataFrame(pd.read_csv('order_brush_order.csv'))

In [0]:
df_order

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [0]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   orderid     222750 non-null  int64 
 1   shopid      222750 non-null  int64 
 2   userid      222750 non-null  int64 
 3   event_time  222750 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


In [0]:
num_order = df_order.shape[0]

num_seller = len(df_order['orderid'].unique())

print('Number of orders:', num_order)
print('Number of unique shops:', num_seller)

Number of orders: 222750
Number of unique shops: 222750


In [0]:
# Grouping 
df_order_group = df_order.groupby('shopid')['orderid'].apply(list).to_frame()
df_order_group['userid'] = df_order.groupby('shopid')['userid'].apply(list)
df_order_group['event_time'] = df_order.groupby('shopid')['event_time'].apply(list)

# Group num_order by shopid and num_buyer 
df_order_group['num_order'] = list(map(lambda x: len(x), df_order_group['orderid']))
df_order_group['num_unique_buyer'] = df_order.groupby('shopid')['userid'].nunique()

# if num_order didn't equal to num_unique_buyer that mean some buyer buy more that 2
df_order_group = df_order_group[df_order_group['num_order'] - df_order_group['num_unique_buyer'] >= 1]

In [0]:
df_order_group

,orderid,userid,event_time,num_order,num_unique_buyer
shopid,,,,,
10084,"[31446282541417, 31377753982301, 3126554273178...","[123715078, 184840040, 6488128, 73993513, 3328...","[2019-12-31 07:04:42, 2019-12-30 12:02:33, 201...",55,54
10100,"[31219662667629, 31415528979818, 3147112150853...","[495431, 30534780, 136619968, 198765759, 13146...","[2019-12-28 16:07:42, 2019-12-30 22:32:08, 201...",42,39
10151,"[31414025664718, 31255231246682, 3112887694712...","[214925713, 43528657, 36268816, 21575761, 8614...","[2019-12-30 22:07:05, 2019-12-29 02:00:32, 201...",193,188
10159,"[31425048460235, 31458346123017, 3133134606985...","[24869430, 802231, 136820136, 125911190, 10581...","[2019-12-31 01:10:48, 2019-12-31 10:25:47, 201...",99,91
10208,"[31117173986024, 31320198136784, 3137988177059...","[86369812, 91720728, 78503053, 211633164, 7991...","[2019-12-27 11:39:33, 2019-12-29 20:03:18, 201...",182,169
...,...,...,...,...,...
213532450,"[31465587099018, 31403767523372, 3146947223170...","[195312059, 167017691, 195312059, 35728972, 21...","[2019-12-31 12:26:27, 2019-12-30 19:16:07, 201...",6,5
213771012,"[31455467617973, 31457145679021]","[6778053, 6778053]","[2019-12-31 09:37:47, 2019-12-31 10:05:45]",2,1
213900783,"[31331073927863, 31366152111175, 3132822056096...","[35438037, 16699499, 124465026, 208784344, 625...","[2019-12-29 23:04:33, 2019-12-30 08:49:12, 201...",7,6


In [0]:
# Function to identify the supicious buyer
def markSupicious(seq):
    Supicious = []
    for i in seq:
        if seq.count(i) >= 3:
            Supicious.append(i)
    if Supicious == []:
        return 0
    else:
        return sorted(list(set(Supicious)))
    
df_order_group['Mark_buyer'] = list(map(markSupicious, df_order_group['userid']))

In [0]:
df_order_group

,orderid,userid,event_time,num_order,num_unique_buyer,Mark_buyer
shopid,,,,,,
10084,"[31446282541417, 31377753982301, 3126554273178...","[123715078, 184840040, 6488128, 73993513, 3328...","[2019-12-31 07:04:42, 2019-12-30 12:02:33, 201...",55,54,0
10100,"[31219662667629, 31415528979818, 3147112150853...","[495431, 30534780, 136619968, 198765759, 13146...","[2019-12-28 16:07:42, 2019-12-30 22:32:08, 201...",42,39,0
10151,"[31414025664718, 31255231246682, 3112887694712...","[214925713, 43528657, 36268816, 21575761, 8614...","[2019-12-30 22:07:05, 2019-12-29 02:00:32, 201...",193,188,0
10159,"[31425048460235, 31458346123017, 3133134606985...","[24869430, 802231, 136820136, 125911190, 10581...","[2019-12-31 01:10:48, 2019-12-31 10:25:47, 201...",99,91,"[62775725, 214988798]"
10208,"[31117173986024, 31320198136784, 3137988177059...","[86369812, 91720728, 78503053, 211633164, 7991...","[2019-12-27 11:39:33, 2019-12-29 20:03:18, 201...",182,169,[11288307]
...,...,...,...,...,...,...
213532450,"[31465587099018, 31403767523372, 3146947223170...","[195312059, 167017691, 195312059, 35728972, 21...","[2019-12-31 12:26:27, 2019-12-30 19:16:07, 201...",6,5,0
213771012,"[31455467617973, 31457145679021]","[6778053, 6778053]","[2019-12-31 09:37:47, 2019-12-31 10:05:45]",2,1,0
213900783,"[31331073927863, 31366152111175, 3132822056096...","[35438037, 16699499, 124465026, 208784344, 625...","[2019-12-29 23:04:33, 2019-12-30 08:49:12, 201...",7,6,0


In [0]:
# Mark order brushing
Mark_buyer = df_order_group[df_order_group['Mark_buyer'] != 0]
Mark_buyer

,orderid,userid,event_time,num_order,num_unique_buyer,Mark_buyer
shopid,,,,,,
10159,"[31425048460235, 31458346123017, 3133134606985...","[24869430, 802231, 136820136, 125911190, 10581...","[2019-12-31 01:10:48, 2019-12-31 10:25:47, 201...",99,91,"[62775725, 214988798]"
10208,"[31117173986024, 31320198136784, 3137988177059...","[86369812, 91720728, 78503053, 211633164, 7991...","[2019-12-27 11:39:33, 2019-12-29 20:03:18, 201...",182,169,[11288307]
10287,"[31199080184075, 31160740703944, 3109326538833...","[495270, 875954, 101436986, 198097381, 1545368...","[2019-12-28 10:24:41, 2019-12-27 23:45:40, 201...",211,190,"[74488711, 103289694, 198097381, 214226569]"
10402,"[31471217004140, 31075350383695, 3146785379529...","[215413147, 185138818, 65216466, 77819, 203648...","[2019-12-31 14:00:17, 2019-12-27 00:02:30, 201...",13,11,[77819]
10536,"[31193005938624, 31115953007150, 3113916044439...","[153387696, 672345, 672345, 672345, 672345, 67...","[2019-12-28 08:43:25, 2019-12-27 11:19:13, 201...",13,5,[672345]
...,...,...,...,...,...,...
209326384,"[31491123051077, 31483532950205, 3133251414481...","[7794832, 7794832, 88887227, 7794832]","[2019-12-31 19:32:04, 2019-12-31 17:25:32, 201...",4,2,[7794832]
210197928,"[31103266871471, 31103396827994, 3110333467196...","[52867898, 52867898, 52867898, 52867898]","[2019-12-27 07:47:47, 2019-12-27 07:49:56, 201...",4,1,[52867898]
211245313,"[31477256052036, 31228328416550, 3133618963726...","[26716334, 63112942, 2048831, 16871221, 428273...","[2019-12-31 15:40:57, 2019-12-28 18:32:08, 201...",52,45,[46875705]


In [0]:
def OrderBrushing(shopid):
    dict_shop_user = dict(zip(df_order_group.index, df_order_group['Mark_buyer']))
    users = dict_shop_user[shopid]
    fraud = []
    for user in users:
        list_time = sorted(list(np.array(df_order_group.loc[shopid].event_time)[np.array(df_order_group.loc[shopid].userid) == user]))
        timestamp = []
        for t in list_time:
            timestamp.append(datetime.strptime(t,'%Y-%m-%d %H:%M:%S'))
        diff_time = []
        for i in range(len(timestamp)-2):
            diff_time.append((timestamp[i+2] - timestamp[i]).seconds)
        diff_time = np.array(diff_time)
        if sum(diff_time < 3600):
            fraud.append(user)
    fraud = '&'.join([str(i) for i in sorted(fraud)])
    if fraud == '':
        return 0
    else:
        return fraud

In [0]:
Supicious_buyer = list(map(OrderBrushing, Mark_buyer.index))

In [0]:
Supicious_shop = pd.DataFrame(Mark_buyer.index)
Supicious_shop['userid'] = Supicious_buyer
Supicious_shop.index = Supicious_shop.shopid
Supicious_shop.index = range(Supicious_shop.shape[0])

In [0]:
Supicious_shop

,shopid,userid
0,10159,62775725&214988798
1,10208,11288307
2,10287,74488711&198097381&214226569
3,10402,77819
4,10536,672345
...,...,...
823,209326384,0
824,210197928,52867898
825,211245313,0
826,212089630,105664365


In [0]:
# Mearge 

df_order_group = df_order.groupby('shopid')['orderid'].apply(list).to_frame()
df_order_group['userid'] = df_order.groupby('shopid')['userid'].apply(list)
df_order_group['event_time'] = df_order.groupby('shopid')['event_time'].apply(list)

df_order_group['num_order'] = list(map(lambda x: len(x), df_order_group['orderid']))

df_order_group['num_unique_buyer'] = df_order.groupby('shopid')['userid'].nunique()

In [0]:
df_order_group = df_order_group['userid'].to_frame()
df_order_group

,userid
shopid,
10009,[196962305]
10051,"[48600461, 2854032]"
10061,"[62464559, 194819216, 168750452, 130633421]"
10084,"[123715078, 184840040, 6488128, 73993513, 3328..."
10100,"[495431, 30534780, 136619968, 198765759, 13146..."
...,...
214662358,[194332514]
214949521,"[46269178, 46269178]"
214964814,[200983383]


In [0]:
df_order_group.userid = 0
df_order_group

,userid
shopid,
10009,0
10051,0
10061,0
10084,0
10100,0
...,...
214662358,0
214949521,0
214964814,0


In [0]:
Supicious_shop.index = Supicious_shop['shopid']

In [0]:
Supicious_shop

,shopid,userid
shopid,,
10159,10159,62775725&214988798
10208,10208,11288307
10287,10287,74488711&198097381&214226569
10402,10402,77819
10536,10536,672345
...,...,...
209326384,209326384,0
210197928,210197928,52867898
211245313,211245313,0


In [0]:
for shop in Supicious_shop['shopid']:
    df_order_group.loc[shop] = Supicious_shop.loc[shop]

In [0]:
df_order_group

,userid
shopid,
10009,0
10051,0
10061,0
10084,0
10100,0
...,...
214662358,0
214949521,0
214964814,0


In [0]:
df_order_group.to_csv('summit_late.csv')